### 从 atomate MongoDB 中获取数据并导出成 json 文件

In [2]:
from atomate.vasp.database import VaspCalcDb

db_json_fn = "db.json"
atomate_db = VaspCalcDb.from_db_file(db_json_fn)

In [3]:
query = {
    "tags.structure_id": "ICET-Training-No-0076857",
    "nsites": 8,
}

count = atomate_db.collection.count_documents(query)
count

4

In [4]:
results = atomate_db.collection.find(query)
results

In [5]:
type(results)  # 游标对象

pymongo.cursor.Cursor

In [6]:
import json

with open("output.json", "w") as file:
    # 将每个 document 转换为 JSON 格式并写入文件
    for result in results:
        # ensure_ascii=False 允许非 ASCII字符在字符串中直接显示
        # default=str 将不是 JSON 序列化的对象转换为字符串
        json_data = json.dumps(result, ensure_ascii=False, default=str)
        file.write(json_data + "\n")


In [7]:
with open("output.json", "r") as file:
    for line in file:
        # 读取每一行，并将其转换为字典
        data = json.loads(line)

    print(data.keys())

dict_keys(['_id', 'dir_name', 'analysis', 'calcs_reversed', 'chemsys', 'completed_at', 'composition_reduced', 'composition_unit_cell', 'custodian', 'elements', 'formula_anonymous', 'formula_pretty', 'formula_reduced_abc', 'input', 'last_updated', 'nelements', 'nsites', 'orig_inputs', 'output', 'run_stats', 'schema', 'state', 'tags', 'task_id', 'task_label', 'transformations'])


循环遍历游标（Cursor）时，游标会随着每次迭代逐步前进，直到遍历完所有文档。游标遍历完毕后，它的位置会停留在结果集的末尾。因此，如果尝试再次遍历同一个游标，循环体内的代码将不会执行，因为游标已经处于结果集的末尾，相当于已经"耗尽"了。

In [8]:
for result in results:
    print(result)

将 results 存储在列表中，可能会占用较多的内存，尤其是当处理大量数据时。在数据集合较小的情况下效果最好。

In [9]:
results = atomate_db.collection.find(query)
results_list = list(results)
len(results_list)

4

In [10]:
# find_one() 返回查询结果的第一个文档
result_one = atomate_db.collection.find_one(query)
type(result_one)

dict